# An input file is generated for ASAXS data correction

Read all files .dat files from a directory
Files sould be ASCII files containing data from ASAXS experiments \
An output file is created which contains information from all files in the directory. \
\
The entries to read from the files are defined in >>>deviceList<<< \
Entries to latere set by hand are defined in >>>deviceByHandList<<< \
\
The following information must be entered concerning the files: \
The file directory \
The file data type like .dat \
The length of the data header \
The structure of entries in the header (seperator, digits to skip after entry \

## Libraries

In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import optimize
import scipy.constants as con
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, LinearSegmentedColormap

## Do the job
First the nessesary information

In [10]:
# give the methadada directory
data_dir = 'files\Data_for_ASAXS'

# give the target file tyle:
target_file_type = '.dat'

# give the output file name
outputFileName = data_dir.split(os.sep)[-1] + '_ASCII_out.txt'

# Concerning the header
# Number of header line
header_lines = 375
# Structure of information in header
seperatorBeforeInfo = '\t'
seperatorAfterInfo = '\t = '
digitsAfterIdentifier = 12 # this is the length of the entry after the identifier you are searching for


# List of experimental divices/information nessesary for the data reduction
deviceList = ['Time','VacSampleX', 'VacSampleY','Energy','Pilatus_Tiff','Pilatus_Trigger','Pilatus_filename','PilatusAcqTime','Keysight1','Keysight2','Keysight3','Keysight4','Keysight1:StandardDeviation','Keysight2:StandardDeviation','Keysight3:StandardDeviation','Keysight4:StandardDeviation','PilatusPos']
# List of values to set by hand
deviceByHandList = ['SampleName','Empty_Index','Background_Index','MaskFile']

Now the iteration an export

In [12]:
# scan the directory
fileList = list()
for entry in os.scandir(data_dir):
    if entry.path.endswith(target_file_type) and entry.is_file():
        fileList.append(entry.path)
# make an path iterator from the found files
fileIterator = iter(fileList)
methaFrame=pd.DataFrame()

for element in fileIterator:
    path_to_data = element
    print(element)
    dF = pd.read_csv(path_to_data, sep='\t',header=header_lines)
    dF = dF.drop([0]) # drop the NaN lines
    # suggesting there is a '# ' directly before the first header entry, drop it
    dfInitialColumnName = dF.columns[0]
    dfNewColumnName = dF.columns.tolist()[0][2:]
    dF = dF.rename(columns={dfInitialColumnName: dfNewColumnName}) # get rid of the '# ' in the first column name
    list_dF_columns = dF.columns.values.tolist()
    
    deviceIterator = iter(deviceList)
    device = next(deviceIterator)
    if device in list_dF_columns:
        deviceFrame = pd.DataFrame({device:dF[device]})
        #print(device)
    else:
        print('Error: First device is not in experimental data:',element)
    
    
    for device in deviceIterator:
        if device in list_dF_columns:
            deviceFrame = deviceFrame.join(dF[device])
        else:
            searchString = seperatorBeforeInfo + device + seperatorAfterInfo
            InfoStringLength = len(device) + len(seperatorAfterInfo)
        
            dataFile = open(path_to_data, 'r')
            resultPosition = dataFile.read().find(searchString)
        
            if resultPosition != -1:
                dataFile = open(path_to_data, 'r')
                longText = dataFile.read(resultPosition+InfoStringLength+digitsAfterIdentifier)
                result = longText[resultPosition+InfoStringLength:]
                deviceFrame[device]=float(result)
                print('Device >>>', device, '<<< is taken as constant from the header with a value of',float(result))
            else:
                deviceFrame[device]='NaN'
                print('Error: Device >>>',device, '<<< not in the data and not in header taken as NaN :(')
                
    methaFrame = methaFrame.append(deviceFrame)

methaFrame = methaFrame.reset_index(drop=True)
for device in deviceByHandList:
    methaFrame[device] = 'NaN'

out_path = data_dir + os.sep + outputFileName
methaFrame.to_csv(out_path)

files\Data_for_ASAXS\fcm_2020kw29tg14_00091.dat
Device >>> Energy <<< is taken as constant from the header with a value of 7000.002692
files\Data_for_ASAXS\fcm_2020kw29tg14_00093.dat
Device >>> Energy <<< is taken as constant from the header with a value of 8003.995109
files\Data_for_ASAXS\fcm_2020kw29tg14_00095.dat
Device >>> Energy <<< is taken as constant from the header with a value of 8238.985877
files\Data_for_ASAXS\fcm_2020kw29tg14_00097.dat
Device >>> Energy <<< is taken as constant from the header with a value of 8303.985331
files\Data_for_ASAXS\fcm_2020kw29tg14_00099.dat
Device >>> Energy <<< is taken as constant from the header with a value of 8323.985784
files\Data_for_ASAXS\fcm_2020kw29tg14_00101.dat
Device >>> Energy <<< is taken as constant from the header with a value of 8329.990438
files\Data_for_ASAXS\fcm_2020kw29tg14_00103.dat
Device >>> Energy <<< is taken as constant from the header with a value of 8973.015959
files\Data_for_ASAXS\fcm_2020kw29tg14_00104.dat
Device 

In [14]:
methaFrame

,Time,VacSampleX,VacSampleY,Energy,Pilatus_Tiff,Pilatus_Trigger,Pilatus_filename,PilatusAcqTime,Keysight1,Keysight2,...,Keysight4,Keysight1:StandardDeviation,Keysight2:StandardDeviation,Keysight3:StandardDeviation,Keysight4:StandardDeviation,PilatusPos,SampleName,Empty_Index,Background_Index,MaskFile
0,34.827,133.00,-1.100000e-05,7000.002692,378.0,379.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,600.0,2.299318e-06,5.430643e-10,...,2.933571e-07,2.642887e-09,2.695514e-11,2.460055e-14,3.442476e-10,3012.874,NaN,NaN,NaN,NaN
1,639.834,123.00,-4.000000e-06,7000.002692,379.0,380.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,300.0,2.152853e-06,5.432002e-10,...,2.934600e-07,2.735894e-09,2.427762e-11,2.329991e-14,3.704751e-10,3012.874,NaN,NaN,NaN,NaN
2,944.835,113.00,4.163336e-17,7000.002692,380.0,381.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,10.0,1.337529e-06,5.675809e-10,...,2.930907e-07,2.514558e-10,1.686306e-11,1.217504e-14,6.612729e-11,3012.874,NaN,NaN,NaN,NaN
3,963.534,88.16,1.000000e-06,7000.002692,381.0,382.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,600.0,1.448823e-06,5.820030e-10,...,2.918486e-07,7.234393e-09,3.077202e-11,2.014422e-14,1.552321e-09,3012.874,NaN,NaN,NaN,NaN
4,1567.693,83.10,4.163336e-17,7000.002692,382.0,383.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,600.0,9.860936e-07,5.657099e-10,...,2.900036e-07,1.581497e-09,2.458183e-11,2.254677e-14,4.827976e-10,3012.874,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,5776.361,23.00,9.999880e-01,9999.991504,517.0,518.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,300.0,4.484193e-07,5.893668e-10,...,2.949844e-08,5.888315e-10,2.384017e-11,2.067708e-14,3.907890e-11,3012.877,NaN,NaN,NaN,NaN
140,6081.651,23.00,-9.149983e+00,9999.991504,518.0,519.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,300.0,4.445866e-07,6.032399e-10,...,2.948161e-08,5.241410e-10,2.439987e-11,2.123514e-14,3.485907e-11,3012.877,NaN,NaN,NaN,NaN
141,6390.580,13.00,7.799996e+00,9999.991504,519.0,520.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,600.0,4.389022e-07,6.102984e-10,...,2.947835e-08,5.559275e-10,3.592902e-11,2.537102e-14,3.793525e-11,3012.877,NaN,NaN,NaN,NaN
142,6994.894,13.00,5.880000e-04,9999.991504,520.0,521.0,/messung/kmc/daten/2020/20_kw29tg14/pilatus_20...,600.0,4.517064e-07,5.958377e-10,...,2.947241e-08,5.888386e-10,3.080868e-11,2.876408e-14,3.933294e-11,3012.877,NaN,NaN,NaN,NaN
